In [4]:
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import seaborn as sns
sns.set()
import gc
gc.collect()
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
from IPython.display import clear_output
#set_matplotlib_formats('retina')
#from tqdm import tqdm
#tqdm.pandas()
import numpy as np
from pyhive import presto
from datetime import datetime, timedelta
#from bson import ObjectId
#from functools import reduce
from sklearn.cluster import KMeans
#from pymongo import MongoClient
import glob
import warnings
warnings.filterwarnings('ignore')
from datetime import date
import json
import re
#import dtale
#from h3 import h3
# import pandasql as ps

In [30]:
metabase_connection = presto.connect(
        #host='bi-trino-2.serving.data.production.internal',
        #host='bi-trino.serving.data.production.internal',
        #host='bi-presto.serving.data.production.internal',
        #host='prime-trino.serving.data.production.internal',
        host= 'presto-gateway.processing.data.production.internal',#"presto.processing.yoda.run",
        port=80,
        protocol='http',
        catalog ='hive',
        username='praveen.u@rapido.bike')

In [ ]:
samsung,xiaomi - 29th live
vivo,oppo - july 4th

In [ ]:
        select eventtype,data_propagationBatchId,data_riderId,data_orderid,yyyymmdd,
        array_agg(eventtype) over (partition by data_orderid) as event_type_agg,
        concat(data_propagationBatchId,data_riderId,data_orderid) as unique_ping_level_id,
        concat(data_orderid,data_riderId) as unique_id_order_level_id,epoch
        from raw.kafka_dispatch_propagation_immutable
        --where yyyymmdd = cast({dt} as varchar)
        where yyyymmdd = '20230610'

            and data_orderid in (select distinct order_id from order_filtering)
            and data_propagationtype in ('multicast')
            and eventtype in ('rider_acknowledged')

        
        union all
        
        select eventtype,data_propagationBatchId,data_riderId,data_orderid,yyyymmdd,
        array_agg(eventtype) over (partition by data_orderid) as event_type_agg,
        concat(data_propagationBatchId,data_riderId,data_orderid) as unique_ping_level_id,
        concat(data_orderid,data_riderId) as unique_id_order_level_id,epoch
        
        from(
        select eventtype,data_propagationBatchId,data_riderId,data_orderid,yyyymmdd,epoch,
        row_number() over(partition by data_propagationBatchId,data_riderId,data_orderid order by epoch desc) as row,data

        from raw.kafka_dispatch_propagation_immutable
        --where yyyymmdd = cast({dt} as varchar)
         where yyyymmdd = '20230610'

            and data_orderid in (select distinct order_id from order_filtering)
            and data_propagationtype in ('multicast')
            and eventtype in ('rider_busy','order_already_accepted','rider_accept_failed','rider_rejected','rider_accepted','rider_ineligible','rider_unreachable')
            ) base
        where row =1

In [47]:
from datetime import datetime
yyyymmdd = pd.date_range(start='2023-07-06', end='2023-07-10').strftime('%Y%m%d').to_list()

df2 = pd.DataFrame()
for i in range(len(yyyymmdd)):
    dt = yyyymmdd[i]
    query1 = f"""
with order_filtering as 
(
    select distinct order_id ,city_name, 
            CASE 
                WHEN substr(quarter_hour,1,2) IN ('08','09','10','11') THEN 'morning_peak'
                WHEN substr(quarter_hour,1,2) IN ('12','13','14','15','16') THEN 'afternoon'
                WHEN substr(quarter_hour,1,2) IN ('17','18','19','20','21') THEN 'evening_peak'
                WHEN substr(quarter_hour,1,2) IN ('00','01','02','03','04','05','06','07') THEN 'rest_morning'        
                ELSE 'rest_evening' END AS time_period, order_status

    from 
        orders.order_logs_snapshot
    where yyyymmdd = cast({dt} as varchar)
    -- where yyyymmdd = '20230314'
        and service_obj_service_name = 'Auto'
        and (spd_fraud_flag = false OR spd_fraud_flag IS NULL)
        and lower(trim(city_name)) in ('hyderabad','chennai','mumbai','bangalore','pune','delhi','jaipur')
)

, rider_pings as
(
   
        select eventtype,data_propagationBatchId,data_riderId,data_orderid,yyyymmdd,
        concat(data_propagationBatchId,data_riderId,data_orderid) as unique_ping_level_id,
        concat(data_orderid,data_riderId) as unique_id_order_level_id,epoch
        from raw.kafka_dispatch_propagation_immutable
        where yyyymmdd = cast({dt} as varchar)
        -- where yyyymmdd = '20230314'

            and data_orderid in (select distinct order_id from order_filtering)
            and data_propagationtype in ('multicast')
            and eventtype in ('rider_acknowledged')

        
        union all
        
        select eventtype,data_propagationBatchId,data_riderId,data_orderid,yyyymmdd,
        concat(data_propagationBatchId,data_riderId,data_orderid) as unique_ping_level_id,
        concat(data_orderid,data_riderId) as unique_id_order_level_id,epoch
        from(
        select eventtype,data_propagationBatchId,data_riderId,data_orderid,yyyymmdd,epoch,
        row_number() over(partition by data_propagationBatchId,data_riderId,data_orderid order by epoch desc) as row,data

        from raw.kafka_dispatch_propagation_immutable
        where yyyymmdd = cast({dt} as varchar)
        -- where yyyymmdd = '20230314'

            and data_orderid in (select distinct order_id from order_filtering)
            and data_propagationtype in ('multicast')
            and eventtype in ('rider_busy','order_already_accepted','rider_accept_failed','rider_rejected','rider_accepted','rider_ineligible','rider_unreachable')
            ) base
        where row =1
    
)

,ord_immutable as 
(    
    select order_id, captain_id, sum(case when order_status = 'dropped' then 1 else 0 end) as order_status_Flag
        from orders.order_logs_immutable
        where yyyymmdd = cast({dt} as varchar)
        -- where yyyymmdd = '20230314'
        and captain_id is not NULL
        and captain_id != ''
        and order_id in (select distinct order_id from order_filtering)
        GROUP BY 1,2
)

, auto_segments as 
(
    select captain_id, yyyymmdd, performance_segment, consistency
    from datasets_internal.captain_auto_pc_segments_v1
    where yyyymmdd = cast({dt} as varchar)
    -- where yyyymmdd = '20230314'
    and lower(trim(city_name)) in ('hyderabad','chennai','mumbai','bangalore','pune','delhi','jaipur')
    group by 1,2,3,4
)

, expiry_order as
(
    select distinct yyyymmdd, order_id, captain_id, stage_name,issue_cause_actor_primary
    from datasets.g2n_classify_immutable 
    where yyyymmdd = cast({dt} as varchar)
    -- where yyyymmdd = '20230314'
    and order_type = 'auto'
    and lower(trim(city_name)) in ('hyderabad','chennai','mumbai','bangalore','pune','delhi','jaipur')
    and captain_id != ''
    and captain_id is not NULL
    and order_id in (select distinct order_id  from order_filtering)
)
, Background_event as 
(
    Select *
    from(

    Select 
    case when event_props_is_background = 'true' then 'Background' 
         when event_props_is_background = 'false' then 'Foreground' else 'Not_Present' end as background_Flag,
    event_props_city,
    event_props_order_id as order_id,
    event_props_user_id as captain_id,
    event_props_yyyymmdd,
    yyyymmdd,
    concat(event_props_order_id, '_',event_props_user_id) as unique_id,
    epoch,
    row_number() over(partition by event_props_order_id,event_props_user_id order by epoch desc) as row
    from 
    clevertap.captain_mmo_order_received_immutable
    where yyyymmdd = cast({dt} as varchar)
    and event_props_order_id in (select distinct order_id  from order_filtering)


) as t1
where row =1
)

select a.city_name,a.yyyymmdd, case when a.yyyymmdd< '20230629' then '1. Pre' else '2. Post' end as Date_Flag,
DOA_Captain_Flag,background_Flag,
sum(accepted_pings) as Accepted_pings,
sum(accepted_pings+already_accepted_intent_pings ) as Accepted_and_Intent_pings,
sum(total_pings) as Total_pings,
sum(net_orders) as Net_orders,
count(distinct case when net_orders > 0 then data_riderid end) as Net_Captains,
count(distinct case when total_pings  > 0 then data_riderid end) as Gross_Captains,
sum(case when total_pings  > 0 then Login_Hours end) as Login_Hours,
sum(OCARA_orders) as OCARA_orders,
sum(COBRA_orders) as COBRA_orders,
sum(ExpiryM_orders) as ExpiryM_orders,
sum(Aborted_P_orders) as Aborted_P_orders

from(

    select yyyymmdd,city_name ,time_period, performance_segment,consistency, DOA_Captain_Flag,background_Flag,data_riderId,
    count(distinct case when eventtype='rider_busy' then unique_ping_level_id end) as riderBusy_pings,
    count(distinct case when eventtype='rider_rejected' then unique_ping_level_id end) as riderreject_Pings,
    count(distinct case when eventtype='rider_accepted' then unique_ping_level_id end) as accepted_pings,
    count(distinct case when eventtype='order_already_accepted' then unique_ping_level_id end) as already_accepted_pings,
    count(distinct case when eventtype='rider_accept_failed' then unique_ping_level_id end) as already_accepted_intent_pings,
    count(distinct case when eventtype='rider_acknowledged' then unique_ping_level_id end) as total_pings,

    count(distinct case when order_status_Flag >= 1 then data_orderid end) as Net_orders,
    count(distinct case when stage_name='OCARA' then data_orderid end) as OCARA_orders,
    count(distinct case when stage_name='OCARA' and issue_cause_actor_primary='Platform' then data_orderid end) as OCARA_P_orders,
    count(distinct case when stage_name='OCARA' and issue_cause_actor_primary='Customer' then data_orderid end) as OCARA_CustC_orders,
    count(distinct case when stage_name='OCARA' and issue_cause_actor_primary='Captain' then data_orderid end) as OCARA_CapC_orders,

    count(distinct case when stage_name='COBRA' then data_orderid end) as COBRA_orders,
    count(distinct case when stage_name='COBRA' and issue_cause_actor_primary='Platform' then data_orderid end) as COBRA_P_orders,
    count(distinct case when stage_name='COBRA' and issue_cause_actor_primary='Customer' then data_orderid end) as COBRA_CustC_orders,
    count(distinct case when stage_name='COBRA' and issue_cause_actor_primary='Captain' then data_orderid end) as COBRA_CapC_orders,
    
    count(distinct case when stage_name='Expiry(Mapped)' then data_orderid end) as ExpiryM_orders,
    count(distinct case when stage_name='Expiry(Mapped)' and issue_cause_actor_primary='Platform' then data_orderid end) as ExpiryM_P_orders,
    count(distinct case when stage_name='Expiry(Mapped)' and issue_cause_actor_primary='Captain' then data_orderid end) as ExpiryM_CapC_orders,
    
    count(distinct case when stage_name='Aborted' then data_orderid end) as Aborted_P_orders
    


    from(
    
    Select distinct a.* ,
    b.order_status_Flag , c.performance_segment,c.consistency,
    d.stage_name,d.issue_cause_actor_primary,
    f.city_name, f.time_period,
    case when e.captainid is not NULL then '01_Provided_after_forcing' else '02_Rest' end as DOA_Captain_Flag,
    background_Flag
    
    from rider_pings  a 
    left join ord_immutable b on a.data_riderId = b.captain_id and a.data_orderid = b.order_id
    left join auto_segments c on a.yyyymmdd = c.yyyymmdd and a.data_riderId = c.captain_id
    left join expiry_order d on a.yyyymmdd = d.yyyymmdd and a.data_riderId = d.captain_id and a.data_orderid = d.order_id
    left join order_filtering f on a.data_orderid = f.order_id

    left join (select distinct captainid
                from (select * from hive.experiments.Auto_Display_Over_Permission_Not_Provided_tire1_cities_v1 where Service_Level = 'Auto'and CT_city_group in ('Hyderabad','Chennai','Mumbai','Bangalore','Pune','Delhi','Jaipur')) a
                left join hive.experiments.auto_display_over_permission_not_provided_latest_tire1_data_v1 b on a.captainid = b.Identity
                where b.Identity is NULL and a.ct_make in ('Vivo','Oppo')
              ) e on a.data_riderId = e.captainid

    left join Background_event g on a.yyyymmdd = g.yyyymmdd and a.data_riderId = g.captain_id and a.data_orderid = g.order_id
    
    )
    group by 1,2,3,4,5,6,7,8
)a

left join (
        select captain_id,yyyymmdd,sum(total_login_hours) as Login_Hours, sum(idle_hours) as idle_hours
        from datasets.captain_metrics_segments
        where yyyymmdd = cast({dt} as varchar)
        and service_mode = 'auto'
        and lower(trim(city)) in ('hyderabad','chennai','mumbai','bangalore','pune','delhi','jaipur')
        group by 1,2
        ) as b 
        on a.yyyymmdd = b.yyyymmdd and a.data_riderid = b.captain_id 


group by 1,2,3,4,5
   
    """
    print(dt)
    t1 = pd.read_sql_query(query1, metabase_connection)
    df2 = pd.concat([t1, df2], axis=0).reset_index(drop=True)

20230706
20230707
20230708
20230709
20230710


In [48]:
df2

,city_name,yyyymmdd,Date_Flag,DOA_Captain_Flag,background_Flag,Accepted_pings,Accepted_and_Intent_pings,Total_pings,Net_orders,Net_Captains,Gross_Captains,Login_Hours,OCARA_orders,COBRA_orders,ExpiryM_orders,Aborted_P_orders
0,Hyderabad,20230710,2. Post,02_Rest,Background,43118,58643,1827587,23101,11217,26485,367901.762778,0,0,0,0
1,Hyderabad,20230710,2. Post,02_Rest,None,1962,2380,63808,1112,227,4063,29526.106111,0,0,0,0
2,Hyderabad,20230710,2. Post,01_Provided_after_forcing,Background,1698,2232,63497,894,460,1191,15994.588611,0,0,0,0
3,Chennai,20230710,2. Post,02_Rest,Background,23696,27683,738265,12168,6540,17096,195554.167778,0,0,0,0
4,Delhi,20230710,2. Post,02_Rest,Foreground,20730,28250,239198,10839,3911,9974,137210.423056,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,Chennai,20230706,2. Post,02_Rest,Foreground,35738,39900,524450,21032,6781,17577,209464.226667,0,0,0,0
206,Mumbai,20230706,2. Post,02_Rest,Foreground,13091,15969,255356,7448,2724,13165,165047.080278,0,0,0,0
207,Pune,20230706,2. Post,02_Rest,Background,17217,24856,504028,10029,6328,17048,285411.096667,0,0,0,0
208,Chennai,20230706,2. Post,01_Provided_after_forcing,None,8,9,118,5,1,42,250.639722,0,0,0,0


In [39]:
#temp1=df2

In [50]:
v1=pd.concat([n1,df2])

In [51]:
v1.yyyymmdd.unique()

array(['20230702', '20230701', '20230630', '20230629', '20230628',
       '20230627', '20230705', '20230704', '20230710', '20230709',
       '20230708', '20230707', '20230706'], dtype=object)

In [52]:
v1.to_clipboard()

In [ ]:
#temp1.yyyymmdd.unique()

In [40]:
temp1.to_csv('del.csv',index=False)

In [20]:
#df2#.yyyymmdd.unique()

In [17]:
df3 = pd.concat([df1, df2], axis=0).reset_index(drop=True)

In [18]:
df3.columns

Index(['city_name', 'yyyymmdd', 'Date_Flag', 'DOA_Captain_Flag',
       'background_Flag', 'Accepted_pings', 'Accepted_and_Intent_pings',
       'Total_pings', 'Net_orders', 'Net_Captains', 'Gross_Captains',
       'Login_Hours', 'OCARA_orders', 'COBRA_orders', 'ExpiryM_orders',
       'Aborted_P_orders'],
      dtype='object')

In [16]:
df2.to_csv("Forced_display_impact.csv", index=False)